In [51]:
import numpy as np
import os

In [52]:
moduleName = [
  'conv1', 'bn1', 'conv2', 'bn2', 'conv3', 'bn3',
  'inception_3a_1x1_conv', 'inception_3a_1x1_bn',
  'inception_3a_pool_conv', 'inception_3a_pool_bn',
  'inception_3a_5x5_conv1', 'inception_3a_5x5_conv2', 'inception_3a_5x5_bn1', 'inception_3a_5x5_bn2',
  'inception_3a_3x3_conv1', 'inception_3a_3x3_conv2', 'inception_3a_3x3_bn1', 'inception_3a_3x3_bn2',
  'inception_3b_3x3_conv1', 'inception_3b_3x3_conv2', 'inception_3b_3x3_bn1', 'inception_3b_3x3_bn2',
  'inception_3b_5x5_conv1', 'inception_3b_5x5_conv2', 'inception_3b_5x5_bn1', 'inception_3b_5x5_bn2',
  'inception_3b_pool_conv', 'inception_3b_pool_bn',
  'inception_3b_1x1_conv', 'inception_3b_1x1_bn',
  'inception_3c_3x3_conv1', 'inception_3c_3x3_conv2', 'inception_3c_3x3_bn1', 'inception_3c_3x3_bn2',
  'inception_3c_5x5_conv1', 'inception_3c_5x5_conv2', 'inception_3c_5x5_bn1', 'inception_3c_5x5_bn2',
  'inception_4a_3x3_conv1', 'inception_4a_3x3_conv2', 'inception_4a_3x3_bn1', 'inception_4a_3x3_bn2',
  'inception_4a_5x5_conv1', 'inception_4a_5x5_conv2', 'inception_4a_5x5_bn1', 'inception_4a_5x5_bn2',
  'inception_4a_pool_conv', 'inception_4a_pool_bn',
  'inception_4a_1x1_conv', 'inception_4a_1x1_bn',
  'inception_4e_3x3_conv1', 'inception_4e_3x3_conv2', 'inception_4e_3x3_bn1', 'inception_4e_3x3_bn2',
  'inception_4e_5x5_conv1', 'inception_4e_5x5_conv2', 'inception_4e_5x5_bn1', 'inception_4e_5x5_bn2',
  'inception_5a_3x3_conv1', 'inception_5a_3x3_conv2', 'inception_5a_3x3_bn1', 'inception_5a_3x3_bn2',
  'inception_5a_pool_conv', 'inception_5a_pool_bn',
  'inception_5a_1x1_conv', 'inception_5a_1x1_bn',
  'inception_5b_3x3_conv1', 'inception_5b_3x3_conv2', 'inception_5b_3x3_bn1', 'inception_5b_3x3_bn2',
  'inception_5b_pool_conv', 'inception_5b_pool_bn',
  'inception_5b_1x1_conv', 'inception_5b_1x1_bn',
  'dense_layer'
]

In [53]:
# Following the Paper from inception module

# The first three layers are the simple conv layer followed 
    # conv -> Maxpool -> BN -> conv -> conv -> BN -> Maxpool
# Now the inception bloacks starts
    # Block = Layer1 [1x1, 1x1, Maxpool], Layer2 [1x1, 3x3, 5x5, 1x1]
    # 2* [Block -> concat] -> Maxpool     
    # 5 * [Block -> concat] -> Maxpool
    # 2* [Block -> concat] -> Maxpool 
    
# The Facenet model have a slightly different architecture when compared to inception modules
convShape = {
    'conv1': [64, 3, 7, 7],
    'conv2': [64, 64, 1, 1],
    'conv3': [192, 64, 3, 3],
# Block 1
    # Chain 1
    'inception_3a_1x1_conv': [64, 192, 1, 1],
    # Chain 2
    'inception_3a_3x3_conv1': [96, 192, 1, 1],
    'inception_3a_3x3_conv2': [128, 96, 3, 3],
    # Chain 3
    'inception_3a_5x5_conv1': [16, 192, 1, 1],
    'inception_3a_5x5_conv2': [32, 16, 5, 5],
    # Chain 4
    'inception_3a_pool_conv': [32, 192, 1, 1],
   
# Block 2
    # Chain 1
    'inception_3b_1x1_conv': [64, 256, 1, 1],
    # Chain 2
    'inception_3b_3x3_conv1': [96, 256, 1, 1],
    'inception_3b_3x3_conv2': [128, 96, 3, 3],
    # Chain 3
    'inception_3b_5x5_conv1': [32, 256, 1, 1],
    'inception_3b_5x5_conv2': [64, 32, 5, 5],
    # Chain 4
    'inception_3b_pool_conv': [64, 256, 1, 1],
   
# Block 3  [Note: No chain 1 and 4]
    # Chain 2
    'inception_3c_3x3_conv1': [128, 320, 1, 1],
    'inception_3c_3x3_conv2': [256, 128, 3, 3],
    # Chain 3 : 
    'inception_3c_5x5_conv1': [32, 320, 1, 1],
    'inception_3c_5x5_conv2': [64, 32, 5, 5],

# Block 4
    # Chain 1
    'inception_4a_1x1_conv': [256, 640, 1, 1],
    # Chain 2
    'inception_4a_3x3_conv1': [96, 640, 1, 1],
    'inception_4a_3x3_conv2': [192, 96, 3, 3],
    # Chain 3
    'inception_4a_5x5_conv1': [32, 640, 1, 1,],
    'inception_4a_5x5_conv2': [64, 32, 5, 5],
    # Chain 4 
    'inception_4a_pool_conv': [128, 640, 1, 1],

# Block 5
    # Chain 2
    'inception_4e_3x3_conv1': [160, 640, 1, 1],
    'inception_4e_3x3_conv2': [256, 160, 3, 3],
    # Chain 3
    'inception_4e_5x5_conv1': [64, 640, 1, 1],
    'inception_4e_5x5_conv2': [128, 64, 5, 5],

# Block 6
    # Chain 1
    'inception_5a_1x1_conv': [256, 1024, 1, 1],
    # Chain 2
    'inception_5a_3x3_conv1': [96, 1024, 1, 1],
    'inception_5a_3x3_conv2': [384, 96, 3, 3],
    # Chain 3
    'inception_5a_pool_conv': [96, 1024, 1, 1],
  
# Bloack 7
    # Chain 1
    'inception_5b_1x1_conv': [256, 736, 1, 1],
    # Chain 2
    'inception_5b_3x3_conv1': [96, 736, 1, 1],
    'inception_5b_3x3_conv2': [384, 96, 3, 3],
    # Chain 3
    'inception_5b_pool_conv': [96, 736, 1, 1],
  
}

In [54]:
parentPath = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceNet-Inception/"
pathDict = {}
for filename in os.listdir(parentPath): 
    if filename.endswith('.csv'):
        pathDict[filename.replace('.csv', '') ] =  os.path.join(parentPath, filename)
# pathDict

In [55]:
from collections import defaultdict

########## Rough
# moduleName = ['inception_3a_1x1_conv', 'inception_3a_1x1_bn',
#                'inception_3a_3x3_conv1', 'inception_3a_3x3_bn1',
#                'inception_3a_3x3_conv2', 'inception_3a_3x3_bn2',
#                'inception_3a_5x5_conv1', 'inception_3a_5x5_bn1',
#                'inception_3a_5x5_conv2', 'inception_3a_5x5_bn2',
#                'inception_3a_pool_conv', 'inception_3a_pool_bn']

########## Rough


moduleWeightDict = defaultdict(lambda: defaultdict())

for num, name in enumerate(moduleName):
    if 'conv' in name:
        conv_w = np.genfromtxt(pathDict[name + '_w'], delimiter=',', dtype=None)
        conv_w = np.reshape(conv_w, convShape[name])
        # Transpose the weight shape to [filterX, filterY, inpChannels, outChannels]
        conv_w = np.transpose(conv_w, (2, 3, 1, 0))
        conv_b = np.genfromtxt(pathDict[name + '_b'], delimiter=',', dtype=None)
        
        moduleWeightDict[name]['w'] =  conv_w
        moduleWeightDict[name]['b'] =  conv_b
    elif 'bn' in name:
            moduleWeightDict[name]['w'] = np.genfromtxt(pathDict[name + '_w'], delimiter=',', dtype=None)
            moduleWeightDict[name]['b'] = np.genfromtxt(pathDict[name + '_b'], delimiter=',', dtype=None)
            moduleWeightDict[name]['m'] = np.genfromtxt(pathDict[name + '_m'], delimiter=',', dtype=None)
            moduleWeightDict[name]['v'] = np.genfromtxt(pathDict[name + '_v'], delimiter=',', dtype=None)
    elif 'dense' in name:
        dense_w = np.genfromtxt(parentPath+'/dense_w.csv', delimiter=',', dtype=None)
        dense_w = np.reshape(dense_w, (128, 736))
        dense_w = np.transpose(dense_w, (1, 0))
        moduleWeightDict[name]['w'] = dense_w
        moduleWeightDict[name]['b'] = np.genfromtxt(parentPath+'/dense_b.csv', delimiter=',', dtype=None)
    else:
        print ('OOPS! Something seems weird')


In [56]:
moduleWeightDict.keys()

dict_keys(['conv1', 'bn1', 'conv2', 'bn2', 'conv3', 'bn3', 'inception_3a_1x1_conv', 'inception_3a_1x1_bn', 'inception_3a_pool_conv', 'inception_3a_pool_bn', 'inception_3a_5x5_conv1', 'inception_3a_5x5_conv2', 'inception_3a_5x5_bn1', 'inception_3a_5x5_bn2', 'inception_3a_3x3_conv1', 'inception_3a_3x3_conv2', 'inception_3a_3x3_bn1', 'inception_3a_3x3_bn2', 'inception_3b_3x3_conv1', 'inception_3b_3x3_conv2', 'inception_3b_3x3_bn1', 'inception_3b_3x3_bn2', 'inception_3b_5x5_conv1', 'inception_3b_5x5_conv2', 'inception_3b_5x5_bn1', 'inception_3b_5x5_bn2', 'inception_3b_pool_conv', 'inception_3b_pool_bn', 'inception_3b_1x1_conv', 'inception_3b_1x1_bn', 'inception_3c_3x3_conv1', 'inception_3c_3x3_conv2', 'inception_3c_3x3_bn1', 'inception_3c_3x3_bn2', 'inception_3c_5x5_conv1', 'inception_3c_5x5_conv2', 'inception_3c_5x5_bn1', 'inception_3c_5x5_bn2', 'inception_4a_3x3_conv1', 'inception_4a_3x3_conv2', 'inception_4a_3x3_bn1', 'inception_4a_3x3_bn2', 'inception_4a_5x5_conv1', 'inception_4a_5x5_c

In [58]:
moduleWeightDict["conv1"]['w'].shape

(7, 7, 3, 64)

## Load train dataset

In [5]:
import h5py
import numpy as np

dirPath = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceRecognition/train_happy.h5'

train_dataset = h5py.File(dirPath)

In [6]:
np.array(train_dataset)

array(['list_classes', 'train_set_x', 'train_set_y'],
      dtype='<U12')

In [7]:
train_dataset['train_set_x']

<HDF5 dataset "train_set_x": shape (600, 64, 64, 3), type "|u1">

In [9]:
np.array(train_dataset["train_set_y"])

array([0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1,

In [25]:
a = np.array(train_dataset['train_set_x'])[0]
b = np.array(train_dataset['train_set_x'])[3]
c = np.array(train_dataset['train_set_x'])[5]
d = np.array(train_dataset['train_set_x'])[6]
e = np.array(train_dataset['train_set_x'])[7]

In [26]:
# Plot a image from the train data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline

In [ ]:
--> Store per person encoding in DataBase, by passing 
    all the image one by one to the "img_to_encoding" funciton

Steps:
    1. First get a model prepared using Keras, "faceRecoModel"
    2. Write the Triplet loss function 
    3. Add the triplet loss function as an optimization to the network.
    4. Load trained weight from FaceNet "load_weights_from_FaceNet"
    5. For a new image let the network pass throught the FaceNet model and output encoding "img_to_encoding"
    6. Compare the encoding